## Install Libraries

In [ ]:
!pip install -q tensorflow=='2.18.0'

## Import Libraries

In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Dropout, Input, Multiply, Softmax, Layer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf

2025-11-21 15:53:36.589203: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763740416.860420      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763740416.943744      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

## Configuration

In [3]:
# Set random seeds for reproducibility
import os
import random

SEED = 0
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Configure TensorFlow for reproducibility
tf.config.experimental.enable_op_determinism()

# Load Dataset

In [ ]:
df = pd.read_excel("tosin-time-series-data/FNCL.6.5.xlsx")

## Data Preprocessing

In [5]:
df = df.sort_values(by=['Dates']).reset_index(drop=True)

# Calculate price diff and movement
df["price_diff"] = df["FNCL 6.5 2022 FL Mtge"].diff()

def encode_movement(x):
    if x > 0:
        return 1  # up
    elif x < 0:
        return 0  # down
    else:
        return -1  # flat (to be removed)

df["movement"] = df["price_diff"].apply(
    lambda x: encode_movement(x) if pd.notnull(x) else -1
)

# Remove rows with NaN prices and flat movements
df = df[df["FNCL 6.5 2022 FL Mtge"].notna()].reset_index(drop=True)
df = df[df["movement"] != -1].reset_index(drop=True)

print(f"Dataset shape after removing NaNs and flats: {df.shape}")
print(f"Movement distribution - Up (1): {(df['movement']==1).sum()}, Down (0): {(df['movement']==0).sum()}")

Dataset shape after removing NaNs and flats: (640, 4)
Movement distribution - Up (1): 322, Down (0): 318


## Feature Engineering

In [6]:
def create_features(data, lags=[1, 2, 3, 5, 10, 21]):
    """Create lagged features and technical indicators"""
    features_df = data.copy()
    price_col = "FNCL 6.5 2022 FL Mtge"
    
    # Lagged prices
    for lag in lags:
        features_df[f'price_lag_{lag}'] = features_df[price_col].shift(lag)
    
    # Lagged price differences
    for lag in [1, 2, 3, 5]:
        features_df[f'diff_lag_{lag}'] = features_df['price_diff'].shift(lag)
    
    # Rolling statistics
    for window in [5, 10, 21]:
        features_df[f'rolling_mean_{window}'] = features_df[price_col].shift(1).rolling(window).mean()
        features_df[f'rolling_std_{window}'] = features_df[price_col].shift(1).rolling(window).std()
        features_df[f'rolling_min_{window}'] = features_df[price_col].shift(1).rolling(window).min()
        features_df[f'rolling_max_{window}'] = features_df[price_col].shift(1).rolling(window).max()
    
    # Price momentum indicators
    features_df['momentum_5'] = features_df[price_col].shift(1) - features_df[price_col].shift(6)
    features_df['momentum_10'] = features_df[price_col].shift(1) - features_df[price_col].shift(11)
    
    # Rate of change
    features_df['roc_5'] = (features_df[price_col].shift(1) - features_df[price_col].shift(6)) / features_df[price_col].shift(6)
    features_df['roc_10'] = (features_df[price_col].shift(1) - features_df[price_col].shift(11)) / features_df[price_col].shift(11)
    
    # Volatility
    returns = features_df[price_col].pct_change()
    features_df['volatility_5'] = returns.shift(1).rolling(5).std()
    features_df['volatility_10'] = returns.shift(1).rolling(10).std()
    
    # Exponential moving averages
    features_df['ema_5'] = features_df[price_col].shift(1).ewm(span=5, adjust=False).mean()
    features_df['ema_21'] = features_df[price_col].shift(1).ewm(span=21, adjust=False).mean()
    
    # Lagged movements
    for lag in [1, 2, 3, 5]:
        features_df[f'movement_lag_{lag}'] = features_df['movement'].shift(lag)
    
    return features_df

In [7]:
# Create features
df_features = create_features(df)
df_features = df_features.dropna().reset_index(drop=True)

print(f"Dataset shape after feature engineering: {df_features.shape}")
print(f"Number of features: {df_features.shape[1] - 3}")

Dataset shape after feature engineering: (619, 38)
Number of features: 35


In [8]:
# Define horizons
HORIZONS = [1, 5, 21, 63]

# Prepare feature columns
exclude_cols = ['Dates', 'FNCL 6.5 2022 FL Mtge', 'movement', 'price_diff']
feature_cols = [col for col in df_features.columns if col not in exclude_cols]

print(f"Feature columns ({len(feature_cols)}): {feature_cols[:10]}...")

Feature columns (34): ['price_lag_1', 'price_lag_2', 'price_lag_3', 'price_lag_5', 'price_lag_10', 'price_lag_21', 'diff_lag_1', 'diff_lag_2', 'diff_lag_3', 'diff_lag_5']...


## Helper Function for Preparing Data

In [9]:
def create_sequences(X, y, time_steps=10):
    """Create sequences for LSTM"""
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        Xs.append(X[i:(i + time_steps)])
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

## Define Sequence Model and Attention Block

In [10]:
def attention_block(inputs):
    """
    Implements an attention mechanism for the LSTM model.
    """
    attention = Dense(1, activation='tanh')(inputs)
    attention = Softmax(axis=1)(attention)
    attention = Multiply()([inputs, attention])
    return attention

In [11]:
def build_bilstm_attention_model(input_shape):
    inputs = Input(shape=input_shape)
    
    # First BiLSTM layer
    x = Bidirectional(LSTM(128, return_sequences=True))(inputs)
    x = Dropout(0.3)(x)
    
    # Second BiLSTM layer
    x = Bidirectional(LSTM(64, return_sequences=True))(x)
    x = Dropout(0.3)(x)
    
    # Attention mechanism
    x = attention_block(x)
    
    # Third BiLSTM layer
    x = Bidirectional(LSTM(32, return_sequences=False))(x)
    x = Dropout(0.3)(x)
    
    # Dense layers
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(16, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

## Model Training and Evaluation

In [12]:
# Results storage
results = []

# Train and evaluate models for each horizon
for horizon in HORIZONS:
    print(f"HORIZON: {horizon} days ahead")
    
    # Create target
    y_target = df_features['movement'].shift(-horizon)
    
    # Remove NaN values
    valid_idx = y_target.notna()
    X_data = df_features[feature_cols][valid_idx].values
    y_data = y_target[valid_idx].values
    
    # Train-test split (80-20)
    split_idx = int(len(X_data) * 0.8)
    X_train, X_test = X_data[:split_idx], X_data[split_idx:]
    y_train, y_test = y_data[:split_idx], y_data[split_idx:]
    
    print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # BiLSTM with Attention
    time_steps = 15  # Increased time steps for better context
    
    X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train, time_steps)
    X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test, time_steps)
    
    print(f"Sequence shape - Train: {X_train_seq.shape}, Test: {X_test_seq.shape}")
    
    bilstm_model = build_bilstm_attention_model((time_steps, X_train_scaled.shape[1]))
    
    # Enhanced callbacks
    early_stop = EarlyStopping(monitor='loss', patience=15, restore_best_weights=True, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
    
    bilstm_model.fit(
        X_train_seq, y_train_seq,
        epochs=100,
        batch_size=32,
        verbose=1,
        callbacks=[early_stop, reduce_lr]
    )
    
    # Predictions
    y_pred_bilstm = (bilstm_model.predict(X_test_seq, verbose=0) > 0.5).astype(int).flatten()
    
    # Metrics
    acc_bilstm = accuracy_score(y_test_seq, y_pred_bilstm)
    prec_up_bilstm = precision_score(y_test_seq, y_pred_bilstm, pos_label=1, zero_division=0)
    prec_down_bilstm = precision_score(y_test_seq, y_pred_bilstm, pos_label=0, zero_division=0)
    f1_up_bilstm = f1_score(y_test_seq, y_pred_bilstm, pos_label=1, zero_division=0)
    f1_down_bilstm = f1_score(y_test_seq, y_pred_bilstm, pos_label=0, zero_division=0)
    recall_up_bilstm = recall_score(y_test_seq, y_pred_bilstm, pos_label=1, zero_division=0)
    recall_down_bilstm = recall_score(y_test_seq, y_pred_bilstm, pos_label=0, zero_division=0)
    
    print(f"\nAccuracy: {acc_bilstm:.4f}")
    print(f"Precision (Up): {prec_up_bilstm:.4f}, Precision (Down): {prec_down_bilstm:.4f}")
    print(f"Recall (Up): {recall_up_bilstm:.4f}, Recall (Down): {recall_down_bilstm:.4f}")
    print(f"F1-Score (Up): {f1_up_bilstm:.4f}, F1-Score (Down): {f1_down_bilstm:.4f}")
    
    results.append({
        'Horizon': horizon,
        'Model': 'BiLSTM+Attention',
        'Accuracy': acc_bilstm,
        'Precision_Up': prec_up_bilstm,
        'Precision_Down': prec_down_bilstm,
        'Recall_Up': recall_up_bilstm,
        'Recall_Down': recall_down_bilstm,
        'F1_Up': f1_up_bilstm,
        'F1_Down': f1_down_bilstm
    })

HORIZON: 1 days ahead
Train size: 494, Test size: 124
Sequence shape - Train: (479, 15, 34), Test: (109, 15, 34)


2025-11-21 15:53:57.240979: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Epoch 1/100


2025-11-21 15:53:57.599795: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


15/15 ━━━━━━━━━━━━━━━━━━━━ 11s 49ms/step - accuracy: 0.5015 - loss: 0.6944 - learning_rate: 0.0010
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5053 - loss: 0.6922 - learning_rate: 0.0010
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.5221 - loss: 0.6922 - learning_rate: 0.0010
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.4852 - loss: 0.6947 - learning_rate: 0.0010
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5203 - loss: 0.6921 - learning_rate: 0.0010
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5497 - loss: 0.6922 - learning_rate: 0.0010
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5213 - loss: 0.6897 - learning_rate: 0.0010
Epoch 8/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5793 - loss: 0.6936 - learning_rate: 0.0010
Epoch 9/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5170 - loss: 0.6880 - learning_rate: 0.0010
Epoch 10/100
15/15 ━━

...


Accuracy: 0.4537
Precision (Up): 0.4783, Precision (Down): 0.4355
Recall (Up): 0.3860, Recall (Down): 0.5294
F1-Score (Up): 0.4272, F1-Score (Down): 0.4779
HORIZON: 21 days ahead
Train size: 478, Test size: 120
Sequence shape - Train: (463, 15, 34), Test: (105, 15, 34)
Epoch 1/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.4351 - loss: 0.6935 - learning_rate: 0.0010
Epoch 2/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.5196 - loss: 0.6928 - learning_rate: 0.0010
Epoch 3/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5725 - loss: 0.6923 - learning_rate: 0.0010
Epoch 4/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5217 - loss: 0.6924 - learning_rate: 0.0010
Epoch 5/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5292 - loss: 0.6912 - learning_rate: 0.0010
Epoch 6/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5260 - loss: 0.6895 - learning_rate: 0.0010
Epoch 7/100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.50

2025-11-21 15:58:19.236366: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}



Accuracy: 0.6095
Precision (Up): 0.6190, Precision (Down): 0.5952
Recall (Up): 0.6964, Recall (Down): 0.5102
F1-Score (Up): 0.6555, F1-Score (Down): 0.5495
HORIZON: 63 days ahead
Train size: 444, Test size: 112
Sequence shape - Train: (429, 15, 34), Test: (97, 15, 34)
Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 11s 47ms/step - accuracy: 0.5336 - loss: 0.6933 - learning_rate: 0.0010
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5736 - loss: 0.6913 - learning_rate: 0.0010
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5511 - loss: 0.6906 - learning_rate: 0.0010
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5772 - loss: 0.6892 - learning_rate: 0.0010
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.5896 - loss: 0.6852 - learning_rate: 0.0010
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.5905 - loss: 0.6864 - learning_rate: 0.0010
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.574

2025-11-21 15:59:41.892622: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}



Accuracy: 0.5258
Precision (Up): 0.5172, Precision (Down): 0.6000
Recall (Up): 0.9184, Recall (Down): 0.1250
F1-Score (Up): 0.6618, F1-Score (Down): 0.2069


In [13]:
results_df = pd.DataFrame(results)
display(results_df)

,Horizon,Model,Accuracy,Precision_Up,Precision_Down,Recall_Up,Recall_Down,F1_Up,F1_Down
0,1,BiLSTM+Attention,0.486239,0.516129,0.474359,0.280702,0.711538,0.363636,0.569231
1,5,BiLSTM+Attention,0.453704,0.478261,0.435484,0.385965,0.529412,0.427184,0.477876
2,21,BiLSTM+Attention,0.609524,0.619048,0.595238,0.696429,0.510204,0.655462,0.549451
3,63,BiLSTM+Attention,0.525773,0.517241,0.600000,0.918367,0.125000,0.661765,0.206897


In [14]:
# Best models by horizon
# print("Best Model Per Horizon (by Accuracy)")

# for horizon in HORIZONS:
#     horizon_results = results_df[results_df['Horizon'] == horizon]
#     best_model = horizon_results.loc[horizon_results['Accuracy'].idxmax()]
#     print(f"Horizon {horizon}: {best_model['Model']} (Accuracy: {best_model['Accuracy']:.4f}, "
#           f"F1_Up: {best_model['F1_Up']:.4f}, F1_Down: {best_model['F1_Down']:.4f})")